# Fe-C diffusion and elastodiffusivity
Taking data from R.G.A. Veiga, M. Perez, C. Becquart, E. Clouet and C. Domain, Acta mater. **59** (2011) p. 6963 [doi:10.1016/j.actamat.2011.07.048](http://dx.doi.org/10.1016/j.actamat.2011.07.048)

Fe in the body-centered cubic phase, $a_0 = 0.28553\text{ nm}$; C sit at octahedral sites, where the transition states between octahedral sites are represented by tetrahedral sites. The data is obtained from an EAM potential, where $C_{11} = 243\text{ GPa}$, $C_{12}=145\text{ GPa}$, and $C_{44} = 116\text{ GPa}$. The tetrahedral transition state is 0.816 eV above the octahedral site, and the attempt frequency is taken as 10 THz ($10^{13}\text{ Hz}$).

The dipole tensors can be separated into *parallel* and *perpendicular* components; the parallel direction points towards the closest Fe atoms for the C, while the perpendicular components lie in the interstitial plane. For the octahedral, the parallel component is 8.03 eV, and the perpendicular is 3.40 eV; for the tetrahedral transition state, the parallel component is 4.87 eV, and the perpendicular is 6.66 eV.

In [4]:
import sys
sys.path.append('../')
import numpy as np
import onsager.crystal as crystal
import onsager.OnsagerCalc as onsager

Create BCC lattice (lattice constant in nm).

In [7]:
a0 = 0.28553
Fe = crystal.Crystal.BCC(a0, "Fe")
print(Fe)

#Lattice:
  a1 = [-0.142765  0.142765  0.142765]
  a2 = [ 0.142765 -0.142765  0.142765]
  a3 = [ 0.142765  0.142765 -0.142765]
#Basis:
  (Fe) 0.0 = [ 0.  0.  0.]


Add carbon interstitial sites at octahedral sites in the lattice. This code (1) gets the set of symmetric Wyckoff positions corresponding to the single site $[00\frac12]$ (first translated into unit cell coordinates), and then adds that new basis to our Fe crystal to generate a *new* crystal structure, that we name "FeC".

In [14]:
uoct = np.dot(Fe.invlatt, np.array([0, 0, 0.5*a0]))
FeC = Fe.addbasis(Fe.Wyckoffpos(uoct), ["C"])
print(FeC)

#Lattice:
  a1 = [-0.142765  0.142765  0.142765]
  a2 = [ 0.142765 -0.142765  0.142765]
  a3 = [ 0.142765  0.142765 -0.142765]
#Basis:
  (Fe) 0.0 = [ 0.  0.  0.]
  (C) 1.0 = [ 0.   0.5  0.5]
  (C) 1.1 = [ 0.5  0.5  0. ]
  (C) 1.2 = [ 0.5  0.   0.5]


Next, we construct a *diffuser* based on our interstitial. We need to create a `sitelist` (which will be the Wyckoff positions) and a `jumpnetwork` for the transitions between the sites. There are tags that correspond to the unique states and transitions in the diffuser. 

In [26]:
chem = 1  # 1 is the index corresponding to our C atom in the crystal
sitelist = FeC.sitelist(chem)
jumpnetwork = FeC.jumpnetwork(chem, 0.6*a0)  # 0.6*a0 is the cutoff distance for finding jumps
FeCdiffuser = onsager.Interstitial(FeC, chem, sitelist, jumpnetwork)
print(FeCdiffuser)

Diffuser for atom 1 (C)
#Lattice:
  a1 = [-0.142765  0.142765  0.142765]
  a2 = [ 0.142765 -0.142765  0.142765]
  a3 = [ 0.142765  0.142765 -0.142765]
#Basis:
  (Fe) 0.0 = [ 0.  0.  0.]
  (C) 1.0 = [ 0.   0.5  0.5]
  (C) 1.1 = [ 0.5  0.5  0. ]
  (C) 1.2 = [ 0.5  0.   0.5]
states:
i:+0.000,+0.500,+0.500
transitions:
i:+0.000,+0.500,+0.500^i:+0.500,+1.000,+0.500



Next, we assemble our data: the energies, prefactors, and dipoles for the C atom in Fe, matched to the *representative* states: these are the first states in the lists, which are also identified by the tags above.